# 2. Продвинутые операции

In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession

## 2.1 Joins и оконные функции

## 2.2 Работа с датами и строками

## 2.3 Оптимизация (партиционирование, кэширование)

## 2.4 UDF (пользовательские функции)

## 2.5 Чтение/запись в разных форматах